# Lesson 3: Reflection and Blogpost Writing

## Setup

In [7]:
# llm_config = {"model": "gpt-3.5-turbo"}

llm_config = {
    "config_list": [
        {
            "model": "llama3.1",  # 
            "api_key": "ollama",  # 
            "base_url": "http://localhost:11434/v1",  
        }
    ],
    "cache_seed": None,  # Turns off caching, useful for testing different models
}

## The task!

In [2]:
task = '''
       Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [15]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [10]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [11]:
import pprint
pprint.pprint(reply)

('"Unlocking AI Mastery: The Power of DeepLearning.AI"\n'
 '\n'
 "In today's data-driven world, artificial intelligence (AI) stands at the "
 'forefront of innovation. But how can one tap into its full potential? Enter '
 'DeepLearning.AI, a platform dedicated to democratizing access to '
 'cutting-edge deep learning knowledge. From introductory tutorials to '
 'advanced courses and community engagement, DeepLearning.AI equips learners '
 'with the skills to succeed in AI development. With top-notch instructors, '
 'engaging interactive content, and a supportive community, this AI powerhouse '
 'ensures that anyone can level up their expertise – making it an invaluable '
 'resource for both enthusiasts and professionals alike.')


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [16]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [17]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


       Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**"Unlocking Human Knowledge: A Closer Look at DeepLearning.AI"**

In the vast realm of artificial intelligence, one entity has been making waves with its comprehensive and freely accessible courses - DeepLearning.AI. Founded by Andrew Ng, a pioneer in AI, this platform offers cutting-edge education in deep learning, enabling enthusiasts and professionals alike to upskill and reskill in AI-related subjects. With engaging video lessons, interactive quizzes, and community-driven forums, DeepLearning.AI has revolutionized the way we learn machine learning concepts. Dive into their courses and unlock a world of possibilities - for free!

--------------------------------------------------------------------------------
Critic (to Writer)

In [18]:
import pprint
pprint.pprint(res.summary)

('**"Unlocking Human Knowledge: A Closer Look at DeepLearning.AI"**\n'
 '\n'
 'Why Collaboration Trumps Competition in AI Education\n'
 '\n'
 'Imagine mastering machine learning concepts through immersive video lessons, '
 'hands-on interactive quizzes, and a vibrant community driven by shared '
 'knowledge – all accessible for free! Welcome to DeepLearning.AI, the '
 'brainchild of Andrew Ng, pioneering educator and engineer at the forefront '
 'of artificial intelligence. This comprehensive platform bridges the gap '
 'between academia and industry, empowering learners worldwide with '
 'cutting-edge education in deep learning.\n'
 '\n'
 "Andrew Ng's legacy shines through in DeepLearning.AI's exceptional quality "
 'of content and interactive features that foster community-driven learning. '
 'Take advantage of this remarkable resource – for free!')


In [20]:
pprint.pprint(res.chat_history[-1]['content'])

('**"Unlocking Human Knowledge: A Closer Look at DeepLearning.AI"**\n'
 '\n'
 'Why Collaboration Trumps Competition in AI Education\n'
 '\n'
 'Imagine mastering machine learning concepts through immersive video lessons, '
 'hands-on interactive quizzes, and a vibrant community driven by shared '
 'knowledge – all accessible for free! Welcome to DeepLearning.AI, the '
 'brainchild of Andrew Ng, pioneering educator and engineer at the forefront '
 'of artificial intelligence. This comprehensive platform bridges the gap '
 'between academia and industry, empowering learners worldwide with '
 'cutting-edge education in deep learning.\n'
 '\n'
 "Andrew Ng's legacy shines through in DeepLearning.AI's exceptional quality "
 'of content and interactive features that foster community-driven learning. '
 'Take advantage of this remarkable resource – for free!')


## Nested chat

In [21]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [22]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [23]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [24]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [ ]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "last_msg",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "last_msg",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "last_msg",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [26]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [27]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


       Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking AI Potential with DeepLearning.AI**

DeepLearning.AI is an online education platform that empowers learners to unlock the power of artificial intelligence (AI). Founded by Andrew Ng, a renowned leader in AI research and teaching, this platform provides in-depth courses, tutorials, and projects on machine learning and deep learning. With its interactive interface, real-world examples, and expert guidance, DeepLearning.AI caters to both beginners and professionals looking to enhance their AI skills. By leveraging its resources, individuals can develop expertise in designing, training, and deploying AI models that drive business growth and innovation.

-----------------------------------------------------------------------

## Get the summary

In [15]:
print(res.summary)

Title: Mastering AI: Your Gateway to DeepLearning.AI

Embark on a transformative journey in artificial intelligence with DeepLearning.AI, a cutting-edge platform reshaping the industry. Curated for all skill levels, from beginners to experts, DeepLearning.AI offers comprehensive courses empowering you with in-depth knowledge of neural networks, deep learning, and more. Elevate your career and skills with expert guidance and hands-on experience. Join the thriving DeepLearning.AI community today and unlock endless opportunities in the AI landscape.

Don't miss out – dive into the world of AI with DeepLearning.AI now!

Incorporate the feedback received from reviewers and deliver a refined version of the blogpost on DeepLearning.AI.
